In [2]:
import numpy as np
import pandas as pd
import os
import tqdm
import itertools
from collections import Counter

import sys
sys.path.append('../')

from util.text_preprocessing import tokenize_comment

# Load Data

## 전체 코멘트 불러오기

In [3]:
total_comments_df = pd.read_pickle("dataset/total-comments_tokenized.pkl")
total_comments_df = total_comments_df.loc[:,['comment_id','comment','comment_token']]
total_comments_df.head(5)

,comment_id,comment,comment_token
0,F3C2DFCFE9E947B19C8A3053DD5C821B,이국종교수는 외과 야전사령관 이다. 그분의 업무에 차질 없도록 물심양면 으로 ...,"[이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없도록, 물, 심양..."
1,61A3B6AAD6124DFFB70EC4275496F8A6,"문재인 대통령님과 이국종 교수님, 그리고 귀순병사 모두 건강하시길 빕니다.","[문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하시길]"
2,B6660EF2C7C14A2F903A08363641CF09,단 한번도 써보지 못했다ㆍ 포인트의 절반만이라도 통신비로 차감을!,"[단, 한번, 보지, 포인트, 절반, 통신비, 차감]"
3,C4F129BFB1FD4543A24A03D96F38351D,우리나라 통신업체들 참 쉽게 돈 벌어요,"[우리나라, 통신, 업체, 돈]"
4,1A21FDDF151E45DF808F0E5B662E8F3C,진짜 쓸데가너무적다.,[진짜]


## 감정 사전 만들기
- 6 emotion : happy, sad, disgust, angry, surprised, fear
- 홍종선, 정연주. (2009). 감정동사의 범주 규정과 유형 분류. 한국어학, 45(), 387-420.

In [4]:
six_emodict_df = pd.read_excel("자료/six_emotion_bigram2_0403.xlsx", encoding = 'utf-8')
six_emodict_df.head()

,happy,sad,disgust,angry,surprised,fear
0,가뿐하다,가슴앓이,가소롭다,갈기갈기,갑작스럽다,가혹하다
1,감개무량하다,가엽다,거북하다,개새끼,경악하다,강압적
2,감격스럽다,가엾다,경박하다,개자식,경이,겁쟁이
3,감격하다,가혹하다,괴상하다,격노하다,급작스럽다,공포감
4,감동스럽다,각박하다,괴팍하다,격분하다,기겁하다,공포스럽다


In [5]:
# 감정 별 단어 개수
six_emodict_df.count()

happy        113
sad          174
disgust       77
angry        110
surprised     47
fear          38
dtype: int64

In [6]:
# 감정 리스트에 변수 할당 
six_emotion = six_emodict_df.columns.tolist()
print(six_emotion)
for emo in six_emotion:
    # 단어 사전에서 emo_words 받아오기
    emo_words = six_emodict_df[emo].dropna().tolist() 
    # 단어 토큰화, 토큰화 후 nan값 된 단어 제거
    emo_words = [tokenize_comment(x)[0] for x in emo_words if len(tokenize_comment(x))>0]
    # 중복 제거
    emo_words = sorted(list(set(emo_words)))
    # 각 감정변수에 차례대로 할당
    globals()[emo] = emo_words
    
angry[:5]

['happy', 'sad', 'disgust', 'angry', 'surprised', 'fear']


['가당찮다', '가증', '갈기갈기', '갈등', '강압']

In [7]:
bi_happy = [('속','시원하다'),('낯','간지럽다'),('정','들다'),('홀','가분')]

bi_sad = [('가슴','아프다'),('기운','없다'),('남','부럽다'),('뒤','숭숭'),('맥','빠지다'),
              ('복','받치다'),('뼈','아프다'),('뼈','저리다'),('애','끓다'),('애','닮다'),
              ('애','타다'),('처','연하다'),('속','썩다'),('속','앓이'),('속','타다')]

bi_disgust =  [('못','밉다'),('정','떨어지다'),('못','마땅하다'),('시','덥다'),('남','부끄럽다'),
                   ('징','그렇다'),('뼈','저리다'),('애','끓다'),('애','닮다'),('처','연하다'),
                   ('속','썩다'),('속','앓이'),('속','타다')]

bi_angry = [('못','되다'),('삐','딱하다'),('비','딱하다'),('약','오르다'),('치','떨리다'),
                ('치밀','오르다'),('성','나다'),('못','마땅하다'),('정','떨어지다'),('남','부끄럽다')]

bi_surprised = [('기상','천외'),('얼','떨다')]

bi_fear = [('뒤','숭숭')]

# 코멘트 데이터에 감정 레이블 달기 

In [8]:
test = ["가슴 아프다 정말", #- sad
        "요즘 상황이 뒤숭숭하다",#sad fear
        "어어어어엉 우웅우 감동스럽다"]#happy]
        
test_token = []
for i in test:
    test_token.append(tokenize_comment(i))
print(test_token)

[['가슴', '아프다', '정말'], ['요즘', '상황', '뒤', '숭숭'], ['감동', '스럽다']]


## 댓글에 특정 감정에 해당하는 단어가 등장하면 update

- bigram
- unigram

In [9]:
total_comments_df['comment_token'][:10]

0    [이국, 종, 교수, 외과, 야전, 사령관, 분, 업무, 차질, 없도록, 물, 심양...
1             [문재인, 대통령, 이국, 종, 교수, 귀순, 병사, 모두, 건강하시길]
2                        [단, 한번, 보지, 포인트, 절반, 통신비, 차감]
3                                    [우리나라, 통신, 업체, 돈]
4                                                 [진짜]
5                                             [포인트, 돈]
6                                   [사법, 적폐, 청산, 시급한듯]
7                 [자하, 산다, 죽음, 사선, 그대, 대한민국, 에세, 아름다운]
8    [당시, 우리, 국군, 미군, 대응, 아주, 공동, 경비, 구역, 모든, 작전, 상...
9                             [우리, 국방부, 대응, 문제, 언론, 임]
Name: comment_token, dtype: object

In [10]:
emo_set = [happy, sad, disgust, angry, surprised, fear]
emo_bigram_set = [bi_happy, bi_sad, bi_disgust, bi_angry, bi_surprised, bi_fear]

check_token_lst = []
emo_cnt_lst = []

total_num = len(total_comments_df)

# comment 돌기
for i,comment in enumerate(total_comments_df['comment_token']):
    print(i/total_num,end="\r")
    check_token = []
    emo_cnt = [0,0,0,0,0,0]
    
    # bigram 처리
    for i, biwords in enumerate(emo_bigram_set):
        for a, b in biwords:
             if (a in comment) & (b in comment):
                check_token.append(a+b)
                emo_cnt[i] += 1 

    # unigram 처리
    for token in comment:
        for i, words in enumerate(emo_set):
            if token in words:
                emo_cnt[i] += 1
                check_token.append(token)

    check_token_lst.append(check_token)
    emo_cnt_lst.append(emo_cnt)

In [11]:
# 비율로 환산
sum_of_emo = np.array(emo_cnt_lst).sum(axis=1,keepdims=True)

emo_ratio = np.array(emo_cnt_lst) / sum_of_emo
emo_ratio[np.isnan(emo_ratio)] = 0

emo_df = pd.DataFrame(emo_ratio,columns = ['happy','sad','disgust','angry','surprised','fear'])

/Users/kim-woojeong/miniconda/envs/python_workspace/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


In [36]:
comment_with_emo = pd.concat([emo_df,
                              total_comments_df,
                              pd.DataFrame({"check":pd.Series(check_token_lst)})],
                             sort=False,axis=1)
# 최소 1개의 감정단어 등장한 댓글만 뽑기
comment_with_emo = comment_with_emo[comment_with_emo['check'].apply(len) > 0]
print(comment_with_emo.shape)
comment_with_emo.head()

(400917, 10)


,happy,sad,disgust,angry,surprised,fear,comment_id,comment,comment_token,check
6,0.0,0.0,0.0,1.0,0.0,0.0,0DC01BDF77744B58957E7A39F3E98A7B,사법 적폐 청산이 시급한듯 싶네요!,"[사법, 적폐, 청산, 시급한듯]",[적폐]
17,0.0,0.0,0.0,1.0,0.0,0.0,194B2B0BB3A642F1AD0670EED8BFCE46,"공동경비구역에서 교전이 벌어지면 뭐가 좋을까? 개무리, 자위당 놈들 비양거리는 소리...","[경비, 구역, 교전, 좋을까, 개, 무리, 자위, 놈, 비양, 거리, 소리]",[놈]
21,0.0,0.0,0.0,0.0,0.0,1.0,22C76F3161F242A59AF956299EBC41EB,연평도 K9은 북한의 포 공격이 끝나고 난 후에 대응했다. 그것도 잘못인가? 전방부...,"[연평도, 은, 북한, 포, 공격, 난, 후, 대응, 그것, 잘못, 방부, 우리, ...",[공포]
42,0.0,0.0,0.0,0.5,0.5,0.0,38C1F648BB99420C8B8CE1D46461EACF,경고성명? 눈하나 깜짝 안 합니다! 같이 맞대응 사격을 했어야했습니다. 저 CX 들...,"[경고, 성명, 눈, 하나, 깜짝, 대응, 사격, 얼마나, 정전협정, 무시, 극단,...","[깜짝, 무시]"
48,0.0,0.0,0.0,1.0,0.0,0.0,C96DC813035A4D3A89317E3433B1DBD2,대한민국의 적폐세력들 : 개정일에게 퍼준 인간들 이를 지금도 따르는 인간들 인터...,"[대한민국, 적폐, 세력, 개정, 일, 퍼준, 인간, 지금, 인터넷, 댓글, 횟수,...",[적폐]


2,393,070 개의 댓글 중 40만 개의 댓글이 감정 단어를 보유

.... 6개 중 1개 꼴

In [37]:
comment_with_emo['check'].apply(len).max()

8

In [38]:
comment_with_emo[comment_with_emo['check'].apply(len)==8]

,happy,sad,disgust,angry,surprised,fear,comment_id,comment,comment_token,check
517667,0.125,0.125,0.375,0.375,0.0,0.0,4412ACD1918C416AB7C71A1D4B9C07F1,네티즌들은 언제부턴가 권력적이고 보수적이 되어버렸다. 자신들보다 어리숙해보이고 도덕...,"[네티즌, 언제, 권력, 보수, 자신, 어리숙해, 보이, 도덕, 보이지, 버는것, ...","[혐오, 증오, 증오, 울분, 울분, 불만, 불만, 기분]"
2318905,0.000,0.000,0.375,0.625,0.0,0.0,F028851C98E34ACBB521E074B13A92CD,"공부를 못하면 개돼지 소리를 듣고, 공부를 잘하면 개돼지 소리를 내뱉는 놈이 된다....","[공부, 개돼지, 소리, 놈, 나라, 교육, 완전히, 자사고, 외고, 폐지, 문제,...","[놈, 새끼, 혐오, 모욕, 증오, 증오, 불만, 불만]"


## 감정 단어 보유 비율에 따라 labeling

In [39]:
def label_with_major_emotion(df, ratio):
    over_index = np.where((df.happy > ratio) |
                          (df.sad > ratio) |
                          (df.disgust > ratio) |
                          (df.angry > ratio) |
               hhhyyy/            (df.surprised > ratio) |
                          (df.fear > ratio))[0]

    over_df = df.iloc[over_index.tolist(),:]

    # Skipna = True will skip all the Na values 
    # find maximum along column axis 
    over_df['emotion'] = over_df.iloc[:,:6].idxmax(axis = 1, skipna = True)
    print(Counter(over_df.emotion))
    
    return over_df

In [41]:
label_with_major_emotion(comment_with_emo, 0.5).to_pickle("dataset/comment_with_emo_over0.5.pkl")

/Users/kim-woojeong/miniconda/envs/python_workspace/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Counter({'angry': 234772, 'sad': 73216, 'happy': 32693, 'disgust': 12806, 'surprised': 7036, 'fear': 5636})
